# Data Processing: 
## A packaget that holds classes and function for manipulating data for automating ETL's report generating

In [18]:
import openpyxl as xl
import pandas as pd
from statistics import mean
from datetime import datetime,timedelta

In [19]:
AGGREGATION_FUNCTIONS = ['mean','max','min','sum','count']

In [25]:
def pivot_table_data(worksheet,primary_keys:list,data_column:str)->dict:
    """
    Extracts the data that mimics spreadsheet's privot table.
    -------
    Parameters:
    worksheet(Worksheet): an openpyxl worksheet object.

    primary_keys(list): list of columns names that will be used as primary keys. Must be of max of 2(row,col).

    data_column(str):The column number of the data field for this table.
    """
    #validate arguments
    simple= True
    if len(primary_keys)>2:
        raise ValueError(f"Expected at most 2 primary_keys. Given {len(primary_keys)} primary keys")
    elif len(primary_keys)==2:
        if primary_keys[0] == primary_keys[1]:
            simple = True
        else:
            simple = False
            
    data = dict()

    #search for the given column names:
    cols = worksheet.iter_cols( max_row = 1,values_only=True)
    col_names = list()
    for col in cols:
        col_names.append(col[0])

    
    if data_column in col_names:
        data_column_number = col_names.index(data_column)+1
    else:
        raise ValueError(f"Given data_column  of '{data_column}' does not exist in the sheet.")
    if primary_keys[0] in col_names:
        key_1 = col_names.index(primary_keys[0])+1
    else:
        raise ValueError(f"Given column name of '{primary_keys[0]}' does not exist in the sheet.")
    if not simple:
        if primary_keys[1] in col_names:
            key_2 = col_names.index(primary_keys[1])+1
        else:
            raise ValueError(f"Given column name of '{primary_keys[1]}' does not exist in the sheet.")
            
        
    for row in worksheet.iter_rows(min_row = 2,min_col = 1, max_col = data_column_number):
        if simple:
            key = row[key_1-1].value
        else:
            key = (row[key_1-1].value,row[key_2-1].value)
        if isinstance(key,datetime):
            key  = key.date()
        if key in data:
            data[key].append(row[data_column_number-1].value)
        else:
            data[key] = [row[data_column_number-1].value]
    return data

In [26]:
def text_to_columns(worksheet, column_number, separator=" "):
    """
    Split text into columns
    """
    for i in range(1, worksheet.max_row + 1):
        if isinstance(worksheet.cell(row=i, column= column_number).value,str):
            split_cell = worksheet.cell(row=i, column= column_number).value.split(separator)
        for j in range(len(split_cell)):
            worksheet.cell(row=i,column=column_number+j).value=split_cell[j]

In [27]:
class KPI:
    """
    Represents 3G or 4G KPI object.
    """
    def __init__(self,kpi_name, agg_fun):
        """
            Creates new instances of KPI object, and empty, data container.
        """
        self.kpi_name = kpi_name
        if agg_fun not in AGGREGATION_FUNCTIONS:
            raise ValueError(f"Given aggregation function of '{agg_fun}' is not supported. Supported functions are: {AGGREGATION_FUNCTIONS}.")
        self.agg_fun = agg_fun
        self.data = dict()
   
    def generate_pivot_table(self, data):
        """
            Adds the data of this KPI based on the agg_fun. The data parameter must be dictionary, generated
            by pivot_table_data function.
        """
        if self.agg_fun == 'mean':
            for key in data:
                self.data[key] = mean(data[key])
        elif self.agg_fun == 'max':
            for key in data:
                self.data[key] = max(data[key])
        elif self.agg_fun == 'min':
            for key in data:
                self.data[key] = min(data[key])
        elif self.agg_fun == 'sum':
            for key in data:
                self.data[key] = sum(data[key])
        elif self.agg_fun == 'count':
            for key in data:
                self.data[key] = len(data[key])

## Testing Script

In [28]:
#wb = xl.load_workbook('Attach KPIs_20250331093842_1.xlsx')

In [29]:
#ws = wb['sheet1']

In [15]:
#kpi = KPI ('Maximum number of attached subscribers(UMTS)', 'max')

In [30]:
#data = pivot_table_data(ws, ['Start'],kpi.kpi_name)

In [36]:
#print(list(data.keys()))

[datetime.date(2025, 3, 22), datetime.date(2025, 3, 23), datetime.date(2025, 3, 24), datetime.date(2025, 3, 25), datetime.date(2025, 3, 26), datetime.date(2025, 3, 27), datetime.date(2025, 3, 28), datetime.date(2025, 3, 29), datetime.date(2025, 3, 30), datetime.date(2025, 3, 31)]


In [37]:
#kpi.generate_pivot_table(data)